## bitslicing

the warp cipher [detail](warp.ipynb).

In [2]:
def integer_to_bitslicing(b0):
    b0_b = bin(b0)[2:].zfill(128)
    R0_b = ""
    R1_b = ""
    R2_b = ""
    R3_b = ""
    for i in range(0, len(b0_b), 4):
        R0_b += b0_b[i]
        R1_b += b0_b[i + 1]
        R2_b += b0_b[i + 2]
        R3_b += b0_b[i + 3]

    return int(R0_b, 2), int(R1_b, 2), int(R2_b, 2), int(R3_b, 2)


def bitslicing_to_integer(R0, R1, R2, R3):
    R0_b = bin(R0)[2:].zfill(32)
    R1_b = bin(R1)[2:].zfill(32)
    R2_b = bin(R2)[2:].zfill(32)
    R3_b = bin(R3)[2:].zfill(32)
    b0_b = ""
    for i in range(0, len(R0_b)):
        b0_b += R0_b[i]
        b0_b += R1_b[i]
        b0_b += R2_b[i]
        b0_b += R3_b[i]
    return int(b0_b, 2)

b0 = 0x0123456789abcdef0123456789abcdef
r0, r1, r2, r3 = integer_to_bitslicing(b0)
b = bitslicing_to_integer(r0, r1, r2, r3)
assert b == b0

## sbox 

$$\begin{align}
    T_{0}  & =  X_{0} \land X_{1}  & T_{1}  & =  X_{0} \land X_{2}  & T_{2}  & =  X_{2} \lor T_{0}   \nonumber       \\
    T_{3}  & =  X_{0} \lor X_{3}   & T_{4}  & =  X_{3} \lor X_{1}   & T_{5}  & =  T_{2} \land T_{3} \nonumber       \\
    T_{6}  & =  T_{2} \oplus T_{0}  & T_{7}  & =  T_{4} \land X_{0}  & T_{8}  & =  X_{1} \lor T_{7}    \nonumber      \\
    T_{9}  & =  T_{7} \oplus X_{1}  & T_{10} & =  \sim T_{0}         & T_{11} & =  T_{4} \oplus T_{0}        \tag{1}          \\
    T_{12} & =  T_{7} \land X_{3}   & T_{13} & =  T_{9} \land T_{3}  & T_{14} & =  T_{6} \lor T_{13}       \nonumber \\
    T_{15} & =  T_{8} \land X_{2}   & T_{16} & = \sim T_{5}          & T_{17} & = \sim T_{3}       \nonumber \\
    T_{18} & =  T_{11} \oplus T_{12} & T_{19} & =  T_{15} \lor T_{17} & Y_{0}  & = T_{16}                 \nonumber    \\
    Y_{1}  & = T_{19}               & Y_{2}  & = T_{18}              & Y_{3}  & = T_{14} \nonumber
\end{align}
$$

In [7]:
S = [0xC, 0xA, 0xD, 0x3, 0xE, 0xB, 0xF, 0x7, 0x8, 0x9, 0x1, 0x5, 0x0, 0x2, 0x4, 0x6]
def sbox(R0, R1, R2, R3):
    r0, r1, r2, r3 = 0, 0, 0, 0
    T = [0] * 20
    # T_{0} =  X_{0} \land X_{1} 
    T[0] = R0 & R1
    # T_{1} =  X_{0} \land X_{2}
    T[1] = R0 & R2
    # T_{2} =  X_{2} \lor T_{0}
    T[2] = R2 | T[0]
    # T_{3} =  X_{0} \lor X_{3} 
    T[3] = R0 | R3
    # T_{4} =  X_{3} \lor X_{1} 
    T[4] = R3 | R1
    # T_{5} =  T_{2} \land T_{3}
    T[5] = T[2] & T[3]
    # T_{6} =  T_{2} \oplus T_{0}
    T[6] = T[2] ^ T[0]
    # T_{7} =  T_{4} \land X_{0} 
    T[7] = T[4] & R0
    # T_{8} =  X_{1} \lor T_{7} 
    T[8] = R1 | T[7]
    # T_{9} =  T_{7} \oplus X_{1}
    T[9] = T[7] ^ R1
    # T_{10} = \sim T_{0}
    T[10] = T[0] ^ 0xFFFF_FFFF
    # T_{11} =  T_{4} \oplus T_{0} 
    T[11] = T[4] ^ T[0]
    # T_{12} =  T_{7} \land X_{3}
    T[12] = T[7] & R3
    # T_{13} =  T_{9} \land T_{3}
    T[13] = T[9] & T[3]
    # T_{14} =  T_{6} \lor T_{13} 
    T[14] = T[6] | T[13]
    # T_{15} =  T_{8} \land X_{2} 
    T[15] = T[8] & R2
    # T_{16} = \sim T_{5} 
    T[16] = T[5] ^ 0xFFFF_FFFF
    # T_{17} = \sim T_{3} 
    T[17] = T[3] ^ 0xFFFF_FFFF
    # T_{18} =  T_{11} \oplus T_{12} 
    T[18] = T[11] ^ T[12]
    # T_{19} =  T_{15} \lor T_{17} 
    T[19] = T[15] | T[17]
    # Y_{0} = T_{16}
    r0 = T[16]
    # Y_{1} = T_{19}
    r1 = T[19]
    # Y_{2} = T_{18}
    r2 = T[18]
    # Y_{3} = T_{14}
    r3 = T[14]

    return r0, r1, r2, r3

b0 = 0x0123456789abcdef0123456789abcdef
r0, r1, r2, r3 = integer_to_bitslicing(b0)
print(f'r0 = {hex(r0)} r1 = {hex(r1)} r2 = {hex(r2)} r3 = {hex(r3)}')
r0, r1, r2, r3 = sbox(r0, r1, r2, r3)
print(f'r0 = {hex(r0)} r1 = {hex(r1)} r2 = {hex(r2)} r3 = {hex(r3)}')
b = bitslicing_to_integer(r0, r1, r2, r3)
b_expected = 0xcad3ebf789150246cad3ebf789150246
assert b == b_expected

r0 = 0xff00ff r1 = 0xf0f0f0f r2 = 0x33333333 r3 = 0x55555555
r0 = 0xeec0eec0 r1 = 0xab13ab13 r2 = 0x5f055f05 r3 = 0x37703770


## permutation 

In [8]:
def right_cyclic_shift(n, bits, shift_amount):
    return ((n >> shift_amount) | (n << (bits - shift_amount))) & ((1 << bits) - 1)


def left_cyclic_shift(n, bits, shift_amount):
    return ((n << shift_amount) | (n >> (bits - shift_amount))) & ((1 << bits) - 1)